In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('/content/fake_news.csv')

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data = data.drop(['id'],axis=1)

In [ ]:
data['content'] = data['author']+' '+data['title']+''+data['text']

In [ ]:
data = data.drop(['author','title','text'],axis=1)

In [ ]:
data.head()

In [ ]:
data = data.fillna('')

In [ ]:
data['content']=data['content'].apply(lambda x: "".join(x.lower()for x in x.split()))

In [ ]:
data['content'] = data['content'].str.replace('[^\w\s]','')

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: "".join(x for x in x.split() if x not in stop))

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content'] = data['content'].apply(lambda x: "".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

In [ ]:
X = data[['content']]
y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',
max_features=5000)
tfidf_vect.fit(data['content'])
xtrain_tfidf = tfidf_vect.transform(X_train['content'])
xtest_tfidf = tfidf_vect.transform(X_test['content'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf = PassiveAggressiveClassifier()
pclf.fit(xtrain_tfidf, y_train)
predictions = pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test, predictions))

In [ ]:
print(metrics.confusion_matrix(y_test, predictions))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlpclf = MLPClassifier(hidden_layer_sizes=(256,64,16),
activation = 'relu',
solver = 'adam')
mlpclf.fit(xtrain_tfidf, y_train)
predictions = mlpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test, predictions))

In [ ]:
def fake_news_det(news):
   input_data = [news]
   vectorized_input_data = tfidf_vect.transform(input_data)
   prediction = pclf.predict(vectorized_input_data)
   print(prediction)

In [ ]:
print(fake_news_det)

In [ ]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

In [ ]:
fake_news_det("""Go to Article
President Barack Obama has been campaigning hard for the woman who is
supposedly going to extend his legacy four more years. The only problem with
stumping for Hillary Clinton, however, is shea€™s not exactly a candidate easy to
get too enthused about. """)

In [ ]:
fake_news_det("""Go to Article
President Barack Obama has been campaigning hard for the woman who is
supposedly going to extend his legacy four more years. The only problem with
stumping for Hillary Clinton, however, is shea€™s not exactly a candidate easy to
get too enthused about. """)

In [ ]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def home():
  return render_template('index.html')

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def home():
  return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
  if request.method == 'POST':
    message = request.form['message']
    pred = fake_news_det(message)
    print(pred)
    return render_template('index.html', prediction=pred)
  else:
    return render_template('index.html', prediction="Something went wrong")

if __name__ == '__main__':
  app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
